In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


In [8]:

# Generate sinusoidal time series data
def generate_sin_wave(freq, num_samples, sample_rate):
    t = np.linspace(0, num_samples / sample_rate, num_samples)
    y = np.sin(2 * np.pi * freq * t)
    return y

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length - 1):
        seq = data[i : (i + seq_length)]
        target = data[i + seq_length]
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)

class TimeSeriesLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.5):
        super(TimeSeriesLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out


In [9]:

# Generate sinusoidal time series data
freq = 2
num_samples = 1000
sample_rate = 100
data = generate_sin_wave(freq, num_samples, sample_rate)

# Prepare the data for the LSTM model
seq_length = 20
X, y = create_sequences(data, seq_length)
X = X.reshape(X.shape[0], X.shape[1], 1)
y = y.reshape(y.shape[0], 1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

# Hyperparameters
input_size = X.shape[-1]
output_size = y.shape[-1]
hidden_size = 64
num_layers = 2
learning_rate = 0.001
num_epochs = 100
dropout_prob = 0.5
weight_decay = 1e-5

# Initialize the model, loss function, and optimizer
model = TimeSeriesLSTM(input_size, hidden_size, num_layers, output_size, dropout_prob=dropout_prob).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train the model
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training completed.")


Epoch [10/100], Loss: 0.4210
Epoch [20/100], Loss: 0.2842
Epoch [30/100], Loss: 0.1401
Epoch [40/100], Loss: 0.0155
Epoch [50/100], Loss: 0.0109
Epoch [60/100], Loss: 0.0115
Epoch [70/100], Loss: 0.0079
Epoch [80/100], Loss: 0.0074
Epoch [90/100], Loss: 0.0057
Epoch [100/100], Loss: 0.0057
Training completed.
